In [36]:
!pip install -q duckduckgo-search  langchain-experimental


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


#### Builtin Tools - DuckDuckGoSearch

In [37]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke('Latest Delhi News')
print(result)

Today's Delhi News: Get all the Latest and Breaking Delhi News on NDTV. Live local Delhi news coverage and updates on Politics, Election, Weather, Crime and More Read the latest Delhi news, headlines & breaking news from Delhi, Delhi crime news, real estate news, politics news, Delhi local news and more at freepressjournal.in Delhi News- Get the latest delhi news including political news, current affairs, delhi Crime news, local news headlines and many more on The Telegraph New Delhi News Updates: Get Latest Breaking News, Photos and Video News on New Delhi. Also find New Delhi latest news headlines, picture gallery, top videos, comments, blog posts and opinion at The Indian Express. Delhi News: Find latest New Delhi news and Headlines from Delhi City. Also check out latest news belongs to Delhi crime, Delhi education news, Delhi real estate, Delhi politics and more


#### Builtin Tools - ShellTool


In [38]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()
result = shell_tool.invoke('ls')
print(result)

Executing command:
 ls
tool-in-langchains.ipynb



/home/nf/Documents/projects-and-notes/LangChain-CampusX/.venv/lib/python3.13/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom tools - Method 1

In [39]:
from langchain_core.tools import tool

In [40]:
# Step-1 Create a function

def multiply(a,b):
    """Multiply two numbers"""
    return a*b

In [41]:
# Step-2 Add type hints
def multiply(a:int,b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [42]:
# Step-3 add a @tool decorator
@tool
def multiply(a:int,b:int) -> int:
    """Multiply two numbers"""
    return a*b


In [43]:
result = multiply.invoke({"a":5, "b": 10})
print(result)

50


In [44]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [45]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Method 2

In [54]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")
        
def multiply_func(a:int, b:int) -> int:
    return a*b

In [55]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [56]:
result = multiply_tool.invoke({"a": 30,"b":20})
print(result)

600


In [57]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}



### Method 3

In [67]:
from langchain.tools import BaseTool
from typing import Type

In [69]:
# args schema using pydantic
class MultiplyInput(BaseModel):
    a : int = Field(required=True, description="The first number to add") 
    b : int = Field(required=True, description="The second number to add") 

In [70]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description : str = "multiply two numbers"
    args_schema : Type[BaseModel] = MultiplyInput
    
    def _run(self, a: int, b: int) -> int:
        return a*b

In [71]:
multiply_tool = MultiplyTool()

result = multiply_tool.invoke({"a":3,"b":10})
print(result)

30


### Toolkit

In [75]:
from langchain.tools import tool

# custom tools
@tool
def add(a:int, b:int) -> int:
    """Add two numbers"""
    return a + b

@tool
def mul(a:int, b:int) -> int:
    """Multiply two numbers"""
    return a * b

In [76]:
class MathToolKit:
    def get_tools(self):
        return [add, multiply]

In [77]:
toolkit = MathToolKit()
tools = toolkit.get_tools()

for tool in tools:
    print("Tool name: ", tool.name)
    print("Tool description: ", tool.description)

Tool name:  add
Tool description:  Add two numbers
Tool name:  multiply
Tool description:  Multiply two numbers
